In [6]:
import pandas as pd

# Initialize the Data

In [7]:
## Crimes Committed in Los Angeles
df = pd.read_csv("Data/Crime_Data_from_2020_to_Present.csv")
## Sun Light Times in Los Angeles
day_df = pd.read_csv("Data/daylightLA.csv")

# Clean and Save Data

In [8]:
## Keep ages from 1 to 100
def clean_age(df):
    return df[df["Vict Age"].between(1, 100)]

## Only keep  the date
def clean_dates(df):
    df['Date Rptd'] = df['Date Rptd'].apply(lambda x: x.split()[0])
    df['DATE OCC'] = df['DATE OCC'].apply(lambda x: x.split()[0])

    return df

## Drop certain columns
def drop_specifics(df, drop_list):
    return df.drop(columns=drop_list)

## Cleaning Data
df = clean_age(df)
df = clean_dates(df)
df = drop_specifics(df, ["LOCATION", "Cross Street", "LAT", "LON"])
df.to_csv("Data/CleanedFile.csv", sep=',', encoding='utf-8', index=False)

# Add Sun Light Times to Data

In [9]:
## Night or Day added to DF
def sun_time(df:pd.DataFrame, day_df:pd.DataFrame):

    day_night_list = []
    for index, row in df.iterrows():

        light:pd.DataFrame = day_df[day_df["Date"] == row["DATE OCC"]]
        if row["TIME OCC"] >= light["Time AM"].item() and row["TIME OCC"] < light["Time PM"].item():
            day_night_list.append("Day")
        else:
            day_night_list.append("Night")

    ## Add Day Light times and whether it's Day or Night
    merged_df = pd.merge(df, day_df, left_on='DATE OCC', right_on='Date', how='left')
    merged_df["Sun"] = day_night_list
    merged_df = drop_specifics(merged_df, ["Date"])

    return merged_df

## Refining Data
df = sun_time(df, day_df)
df.to_csv("Data/RefinedFile.csv", sep=',', encoding='utf-8', index=False)